In [1]:
import sys
import os
from googleapiclient.discovery import build
from typing import List, Literal
from pydantic import BaseModel
from IPython.display import JSON


# import nest_asyncio
# nest_asyncio.apply()

sys.path.append(os.path.abspath(".."))
from src import YouTubeHelper, llm_client

In [2]:
class StockMetric(BaseModel):
    label: str
    value: str

class StockEntry(BaseModel):
    ticker: str
    company_name: str
    sentiment: Literal["Bullish", "Bearish", "Neutral"]
    thesis: str
    sources: List[str]
    metrics: List[StockMetric]

class MarketReport(BaseModel):
    report_title: str
    stocks: List[StockEntry]


In [3]:
with open("../reports/2026-02-04_transcripts.txt", 'r', encoding='utf-8') as file:
    transcript = file.read()

with open("../prompts/combined_transcript_summary.md", 'r', encoding='utf-8') as file:
    system_prompt = file.read()


In [4]:
llm = llm_client.LLMClient(provider="google")
json_report = llm.chat(system_prompt=system_prompt, user_message=transcript, response_schema=MarketReport) 


In [6]:
display(JSON(json_report.model_dump()))

<IPython.core.display.JSON object>